# 回帰モデルの評価指標であるMSE，RMSE，MAE，MAPE，RMSLEについて理解する(2022/02/17)
---

## 概要
---

回帰モデルの評価指標として知られる以下の5つについて調査し，Pythonを用いたシミュレーションを行う．

- MSE(Mean Squared Error)
- RMSE(Root Mean Squared Error)
- MAE(Mean Absolute Error)
- MAPE(Mean Absolute Percentage Error)
- RMSLE(Root Mean Squared Logarithmic Error)

## MSE(Mean Squared Error)
---
MSEは二乗誤差平均とも呼ばれ，データの値と回帰モデルが推定した値との差の二乗の算術平均として与えられる．

\begin{eqnarray}
MSE = \frac{1}{N}\sum_{i=1}^{N}(y_i - \hat{y}_i)^2
\end{eqnarray}

$N$はデータ点の数を表す．$y_i$は目的変数の実現値であり，$\hat{y}_i$は回帰モデルが予測した値である．

MSEの性質として，外れ値やスケールの影響を大きく受けることが挙げられる．残差が大きい場合に，二乗によってさらに強調されるためである．そのため，外れ値がある場合はそれを除いたり，データ点の値をあらかじめ標準化しておく等の工夫が必要である．また，以降に紹介するRMSEやMAEを用いても良いかもしれない．

MAEを算出するために，上記の計算式を直接実装しても良いが，Pythonではscikit-learnに関数が用意されている．

In [1]:
# ボストンの住宅価格データセットを読み込む
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/rasbt/python-machine-learning-book/master/code/datasets/housing/housing.data', header=None, sep='\s+')
df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [2]:
# 重回帰モデルの作成
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X = df.iloc[:,:-1].values
y = df.loc[:,'MEDV'].values

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, shuffle=True)

# 重回帰モデルを訓練
reg = LinearRegression().fit(X_train, y_train)

# 偏回帰係数
reg.coef_

array([-9.85424717e-02,  6.07841138e-02,  5.91715401e-02,  2.43955988e+00,
       -2.14699650e+01,  2.79581385e+00,  3.57459778e-03, -1.51627218e+00,
        3.07541745e-01, -1.12800166e-02, -1.00546640e+00,  6.45018446e-03,
       -5.68834539e-01])

In [3]:
# MSEの算出
from sklearn.metrics import mean_squared_error

y_pred_train = reg.predict(X_train)
y_pred_test = reg.predict(X_test)

print('MSE(train):', mean_squared_error(y_train, y_pred_train))
print('MSE(test):', mean_squared_error(y_test, y_pred_test))

MSE(train): 23.513334449327026
MSE(test): 19.831323672063142


## RMSE(Root Mean Squared Error)
---

RMSEはMSEの平方根をとったものである．

\begin{eqnarray}
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{N}\sum_{i=1}^{N}(y_i - \hat{y}_i)^2}
\end{eqnarray}

MSEでは外れ値などによって値が大きくなる懸念があるが，平方根をとっているため影響が表れにくいという性質がある．

MSEの算出に用いたscikit-learnのメソッドのフラグを変えることで，RMSEも求められる(もちろん，MSEの値の平方根を取るだけでもよい)．

In [4]:
# RMSEの算出
print('RMSE(train):', mean_squared_error(y_train, y_pred_train, squared=False))
print('RMSE(test):', mean_squared_error(y_test, y_pred_test, squared=False))

RMSE(train): 4.849055005805464
RMSE(test): 4.453237437198149


## MAE(Mean Absolute Error)
---

MAEは残差の絶対値をとり，それの算術平均をとることで求められる値である．

\begin{eqnarray}
MAE = \frac{1}{N}\sum_{i=1}^{N}|y_i-\hat{y}_i|
\end{eqnarray}

RMSEと同様に，MSEにあるような外れ値の影響を受けにくい．ただし，これもRMSEと同様にスケールの影響は受ける．

MAEの算出には，scikit-learnのmean_absolute_errorメソッドが便利

In [5]:
# MAEの算出
from sklearn.metrics import mean_absolute_error

print('MAE(train):', mean_absolute_error(y_train, y_pred_train))
print('MAE(test):', mean_absolute_error(y_test, y_pred_test))

MAE(train): 3.3444361206579747
MAE(test): 3.3446655035987467


ここまでの3指標の印象として，残差に過敏に反応するMSEの方が性能差をとらえやすいような気がしている．

## 参考文献
---

[housing.data，rasbt](https://github.com/rasbt/python-machine-learning-book/blob/master/code/datasets/housing/housing.data)

[sklearn.linear_model.LinearRegression，scikit-learn 1.0.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

[sklearn.model_selection.train_test_split，scikit-learn 1.0.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

[sklearn.metrics.mean_squared_error，scikit-learn 1.0.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)

[sklearn.metrics.mean_absolute_error，scikit-learn 1.0.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html)